In [1]:
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
import pandas as pd
import numpy as np

### read data

In [3]:
data = pd.read_parquet('../data/curated/data/')

In [4]:
data['order_datetime'] = data['order_datetime'].astype('str')
data = data[data['order_datetime']<='2022-08-28']

In [5]:
data = data.dropna(subset=['New cases / day'])
data['New cases / day'] = data['New cases / day'].apply(lambda x: float(x.replace(',', '')))
data['postcode'] = data['postcode'].astype('int')
data['total'] = data['total'].astype('float64')
data['merchant_abn'] = data['merchant_abn'].astype('float64')

In [6]:
data = data.sort_values(['merchant_abn', 'order_datetime'])
grouped = data.groupby('merchant_abn')

In [7]:
genders = list(set(list(data.gender)))
states = list(set(list(data.state)))
print(genders)
print(states)

['Undisclosed', 'Female', 'Male']
['VIC', 'ACT', 'NT', 'SA', 'TAS', 'QLD', 'WA', 'NSW']


In [8]:
data.head()

,user_id,merchant_abn,dollar_value,order_id,merchant_name,tags,take_rate,type,postcode,consumer_id,...,state,gender,POA_CODE21,2021_popu,latitude,longitude,New cases / day,month,total,order_datetime
4255,10285,1.002328e+10,311.872869,e41d1fb7-d7b9-4758-9c20-1d34337a312d,Felis Limited,"furniture, home furnishings and equipment shop...",0.18,e,6479,191378,...,WA,Female,6479,243.959797,118.396935,-30.651794,5.0,2021-02-01,30332.5,2021-02-28
5409,714,1.002328e+10,279.618513,2bfa1772-2c93-4aaa-8a32-7e393f2c4267,Felis Limited,"furniture, home furnishings and equipment shop...",0.18,e,3555,642522,...,VIC,Female,3555,21059.999858,144.242651,-36.806383,5.0,2021-02-01,30332.5,2021-02-28
13719,5568,1.002328e+10,110.075256,078410a3-9595-44ba-8714-2effd0e91231,Felis Limited,"furniture, home furnishings and equipment shop...",0.18,e,2154,501304,...,NSW,Female,2154,41084.207790,150.999880,-33.727887,5.0,2021-02-01,30332.5,2021-02-28
24515,10144,1.002328e+10,195.744544,6a9f5c53-b014-4d98-bb5b-90d06263acc3,Felis Limited,"furniture, home furnishings and equipment shop...",0.18,e,5577,474931,...,SA,Female,5577,1774.542610,137.245303,-35.065603,8.0,2021-03-01,30725.9,2021-03-01
25417,23597,1.002328e+10,189.251521,1f73f73e-09c3-484a-a724-c67e32db65fe,Felis Limited,"furniture, home furnishings and equipment shop...",0.18,e,3660,1314310,...,VIC,Male,3660,8067.650238,145.264868,-37.056397,8.0,2021-03-01,30725.9,2021-03-01


In [9]:
all_features = []
for abn, df in grouped:
    feature_list = [abn]
    
    # dollar value
    feature_list.append(np.mean(df.dollar_value))
    feature_list.append(np.min(df.dollar_value))
    feature_list.append(np.max(df.dollar_value))

    # ncd
    feature_list.append(np.mean(df['New cases / day']))

    # gender
    temp = len(df)
    feature_list.extend(len(df[df.gender == g])/temp for g in genders)
    
    # state
    feature_list.extend(len(df[df.state == s])/temp for s in states)
    
    # timeline
    feature_list.append((pd.Timestamp(list(df.iloc[-1:]['order_datetime'])[0]) - pd.Timestamp(list(df.iloc[:1]['order_datetime'])[0])).days)

    # print(feature_list)
    all_features.append(feature_list)

In [10]:
new_data = pd.DataFrame(all_features, columns=['merchant_abn', 'dollar_mean', 'dollar_min', 'dollar_max', 'ncd_mean', \
    genders[0]+'_count', genders[1]+'_count', genders[2]+'_count', \
    states[0]+'_count', states[1]+'_count', states[2]+'_count', states[3]+'_count', states[4]+'_count', states[5]+'_count', \
    states[6]+'_count', states[7]+'_count', 'time_span'])

In [11]:
new_data

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4417,9.997431e+10,286.284404,19.023977,1216.210828,16049.191304,0.147826,0.469565,0.382609,0.252174,0.008696,0.000000,0.113043,0.034783,0.173913,0.121739,0.295652,538
4418,9.997666e+10,150.076506,3.078433,671.566546,17621.841037,0.098736,0.443165,0.458099,0.229977,0.009859,0.015716,0.113524,0.035482,0.149885,0.157938,0.287618,546
4419,9.998791e+10,357.227525,57.274540,1676.478636,21472.900000,0.075000,0.443750,0.481250,0.212500,0.006250,0.012500,0.162500,0.037500,0.150000,0.143750,0.275000,534
4420,9.998904e+10,56552.394443,56552.394443,56552.394443,2103.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0


### merchant data for train y

In [12]:
merchants = pd.read_parquet('../data/curated/merchants_data.parquet')

In [13]:
merchants.head()

,merchant_name,tags,merchant_abn,take_rate,type
0,Felis Limited,"furniture, home furnishings and equipment shop...",10023283211,0.18,e
1,Arcu Ac Orci Corporation,"cable, satellite, and other pay television and...",10142254217,4.22,b
2,Nunc Sed Company,"jewelry, watch, clock, and silverware shops",10165489824,4.40,b
3,Ultricies Dignissim Lacus Foundation,"watch, clock, and jewelry repair shops",10187291046,3.29,b
4,Enim Condimentum PC,"music shops - musical instruments, pianos, and...",10192359162,6.33,a


In [14]:
merchants = merchants[['merchant_abn', 'tags', 'take_rate']]
merchants.head()

,merchant_abn,tags,take_rate
0,10023283211,"furniture, home furnishings and equipment shop...",0.18
1,10142254217,"cable, satellite, and other pay television and...",4.22
2,10165489824,"jewelry, watch, clock, and silverware shops",4.40
3,10187291046,"watch, clock, and jewelry repair shops",3.29
4,10192359162,"music shops - musical instruments, pianos, and...",6.33


In [15]:
merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4026 entries, 0 to 4025
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   merchant_abn  4026 non-null   int64  
 1   tags          4026 non-null   object 
 2   take_rate     4026 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 94.5+ KB


In [16]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
encoder = LabelEncoder()  
tags = encoder.fit_transform(merchants['tags'].values)  
tags = np.array([tags]).T
merchants = pd.concat([merchants, pd.DataFrame(tags)], axis=1)

In [17]:
merchants.head()

,merchant_abn,tags,take_rate,0
0,10023283211,"furniture, home furnishings and equipment shop...",0.18,11
1,10142254217,"cable, satellite, and other pay television and...",4.22,5
2,10165489824,"jewelry, watch, clock, and silverware shops",4.40,15
3,10187291046,"watch, clock, and jewelry repair shops",3.29,24
4,10192359162,"music shops - musical instruments, pianos, and...",6.33,18


In [18]:
merchants = merchants.rename({merchants.columns[-1]:'tag_int'}, axis=1)

- tag prediction - classification

In [110]:
dataset = pd.merge(new_data, merchants[['merchant_abn','tag_int']], 'left', 'merchant_abn')

In [111]:
dataset.dropna(subset=['tag_int'])

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,tag_int
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546,11.0
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546,5.0
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249,15.0
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540,24.0
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4416,9.993898e+10,30.010095,0.005036,332.645347,17703.350356,0.101815,0.449190,0.448995,0.231303,0.009786,0.014582,0.113804,0.036034,0.148347,0.155606,0.290538,546,19.0
4417,9.997431e+10,286.284404,19.023977,1216.210828,16049.191304,0.147826,0.469565,0.382609,0.252174,0.008696,0.000000,0.113043,0.034783,0.173913,0.121739,0.295652,538,4.0
4418,9.997666e+10,150.076506,3.078433,671.566546,17621.841037,0.098736,0.443165,0.458099,0.229977,0.009859,0.015716,0.113524,0.035482,0.149885,0.157938,0.287618,546,20.0
4419,9.998791e+10,357.227525,57.274540,1676.478636,21472.900000,0.075000,0.443750,0.481250,0.212500,0.006250,0.012500,0.162500,0.037500,0.150000,0.143750,0.275000,534,17.0


In [112]:
train_tag = dataset.dropna(subset=['tag_int'])
test_tag = dataset[dataset.tag_int.isna()]

In [113]:
train_tag.tag_int = train_tag.tag_int.astype('int')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [114]:
train_tag.dtypes

merchant_abn         float64
dollar_mean          float64
dollar_min           float64
dollar_max           float64
ncd_mean             float64
Undisclosed_count    float64
Female_count         float64
Male_count           float64
VIC_count            float64
ACT_count            float64
NT_count             float64
SA_count             float64
TAS_count            float64
QLD_count            float64
WA_count             float64
NSW_count            float64
time_span              int64
tag_int                int64
dtype: object

In [115]:
X_train_tag = train_tag.loc[:, :'time_span']

In [116]:
X_train_tag.head(5)

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544


In [117]:
Y_train_tag = train_tag['tag_int']

In [118]:
X_test_tag = test_tag.loc[:, :'time_span']

In [119]:
from sklearn.naive_bayes import MultinomialNB

In [120]:
from sklearn.multiclass import OneVsRestClassifier
model = OneVsRestClassifier(MultinomialNB(alpha=0.1))
multi_model = model.fit(X_train_tag, Y_train_tag)
y_pred_tag = multi_model.predict(X_test_tag)

In [121]:
test_tag['tag_int'] = y_pred_tag

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
test_tag.head()

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,tag_int
29,1.067232e+10,3929.548403,791.291712,9666.915630,17160.922330,0.135922,0.466019,0.398058,0.223301,0.019417,0.009709,0.126214,0.038835,0.135922,0.135922,0.310680,545,22
47,1.102934e+10,2535.748329,281.078519,5755.891571,7239.714286,0.000000,0.571429,0.428571,0.142857,0.000000,0.071429,0.142857,0.142857,0.071429,0.142857,0.285714,510,22
53,1.113751e+10,114.993262,0.309387,894.480908,18284.652199,0.099943,0.444318,0.455740,0.237579,0.007424,0.011993,0.115363,0.035979,0.146202,0.154198,0.291262,546,21
60,1.124043e+10,127.984243,0.024040,1408.151712,18054.871882,0.104082,0.442844,0.453074,0.231931,0.009383,0.013806,0.110344,0.036933,0.148724,0.160422,0.288457,546,18
65,1.135815e+10,11972.309236,1785.170832,37268.557739,20327.285714,0.103896,0.402597,0.493506,0.298701,0.000000,0.025974,0.103896,0.012987,0.129870,0.168831,0.259740,532,22


In [144]:
total_tag = pd.concat([train_tag, test_tag], ignore_index = True)

In [146]:
total_tag.head()

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,tag_int
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546,11
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546,5
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249,15
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540,24
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544,18


In [149]:
len(total_tag)

4422

- rate prediction (linear_prediction)

In [134]:
rate_data = pd.merge(new_data, merchants[['merchant_abn','take_rate']], 'left', 'merchant_abn')

In [135]:
rate_data.dropna(subset=['take_rate'])

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,take_rate
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546,0.18
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546,4.22
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249,4.40
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540,3.29
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544,6.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4416,9.993898e+10,30.010095,0.005036,332.645347,17703.350356,0.101815,0.449190,0.448995,0.231303,0.009786,0.014582,0.113804,0.036034,0.148347,0.155606,0.290538,546,4.50
4417,9.997431e+10,286.284404,19.023977,1216.210828,16049.191304,0.147826,0.469565,0.382609,0.252174,0.008696,0.000000,0.113043,0.034783,0.173913,0.121739,0.295652,538,3.17
4418,9.997666e+10,150.076506,3.078433,671.566546,17621.841037,0.098736,0.443165,0.458099,0.229977,0.009859,0.015716,0.113524,0.035482,0.149885,0.157938,0.287618,546,6.57
4419,9.998791e+10,357.227525,57.274540,1676.478636,21472.900000,0.075000,0.443750,0.481250,0.212500,0.006250,0.012500,0.162500,0.037500,0.150000,0.143750,0.275000,534,6.82


In [136]:
train_rate = rate_data.dropna(subset=['take_rate'])
test_rate = rate_data[rate_data.take_rate.isna()]

In [137]:
X_train = train_rate.loc[:, :'time_span']
Y_train = train_rate['take_rate']
X_test = test_rate.loc[:, :'time_span']

In [138]:
from sklearn.linear_model import LinearRegression

In [139]:
linreg = LinearRegression()

In [140]:
linreg.fit(X_train, Y_train)

LinearRegression()

In [141]:
y_pred = linreg.predict(X_test)

In [142]:
test_rate['take_rate'] = y_pred

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [143]:
test_rate.head()

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,take_rate
29,1.067232e+10,3929.548403,791.291712,9666.915630,17160.922330,0.135922,0.466019,0.398058,0.223301,0.019417,0.009709,0.126214,0.038835,0.135922,0.135922,0.310680,545,4.323630
47,1.102934e+10,2535.748329,281.078519,5755.891571,7239.714286,0.000000,0.571429,0.428571,0.142857,0.000000,0.071429,0.142857,0.142857,0.071429,0.142857,0.285714,510,4.368066
53,1.113751e+10,114.993262,0.309387,894.480908,18284.652199,0.099943,0.444318,0.455740,0.237579,0.007424,0.011993,0.115363,0.035979,0.146202,0.154198,0.291262,546,4.491005
60,1.124043e+10,127.984243,0.024040,1408.151712,18054.871882,0.104082,0.442844,0.453074,0.231931,0.009383,0.013806,0.110344,0.036933,0.148724,0.160422,0.288457,546,4.497794
65,1.135815e+10,11972.309236,1785.170832,37268.557739,20327.285714,0.103896,0.402597,0.493506,0.298701,0.000000,0.025974,0.103896,0.012987,0.129870,0.168831,0.259740,532,4.012999


In [145]:
total_rate = pd.concat([train_rate, test_rate], ignore_index = True)

In [147]:
total_rate.head()

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,take_rate
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546,0.18
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546,4.22
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249,4.40
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540,3.29
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544,6.33


In [148]:
len(total_rate)

4422

In [152]:
total_merchant = pd.merge(total_tag, total_rate[['merchant_abn', 'take_rate']], on='merchant_abn')

In [153]:
total_merchant.head()

,merchant_abn,dollar_mean,dollar_min,dollar_max,ncd_mean,Undisclosed_count,Female_count,Male_count,VIC_count,ACT_count,NT_count,SA_count,TAS_count,QLD_count,WA_count,NSW_count,time_span,tag_int,take_rate
0,1.002328e+10,215.907357,4.016398,1306.809868,17671.272822,0.101045,0.448084,0.450871,0.233798,0.008362,0.014634,0.106969,0.035192,0.148084,0.159582,0.293380,546,11,0.18
1,1.014225e+10,38.887890,0.010086,302.116185,17868.966892,0.104590,0.430399,0.465011,0.243040,0.009782,0.016178,0.103461,0.041761,0.142212,0.162152,0.281415,546,5,4.22
2,1.016549e+10,12785.268849,4050.093665,20636.897020,16378.000000,0.500000,0.250000,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.250000,249,15,4.40
3,1.018729e+10,118.394949,1.007069,544.979881,20445.269360,0.114478,0.451178,0.434343,0.215488,0.006734,0.010101,0.141414,0.043771,0.161616,0.161616,0.259259,540,24,3.29
4,1.019236e+10,459.328629,10.364227,2314.116256,16838.854103,0.085106,0.431611,0.483283,0.231003,0.009119,0.009119,0.112462,0.033435,0.161094,0.164134,0.279635,544,18,6.33
